In [2]:
import os
import pandas as pd

In [21]:
file_path = '../data/Dados-Processo-tratados/todos_ensaios.csv'

df = pd.read_csv(file_path, delimiter=';', on_bad_lines='skip')
df.groupby('ensaio').mean()

,time,sensor_PT100_1,sensor_PT100_3,sensor_PT100_4,sensor_humidity_1,sensor_TH_1,sensor_humidity_2,sensor_TH_2,product_humidity,sensor_PT100_2
ensaio,,,,,,,,,,
1,2700.612500,40.545000,33.667500,31.748750,50.213750,25.898750,65.567500,30.125000,10.353750,NaN
2,2070.833333,34.595000,33.670000,31.265000,77.450000,21.071667,81.498333,28.008333,14.671667,-48.710000
3,2700.125000,40.898750,47.890000,47.186250,30.806250,31.015000,36.581250,42.865000,8.588750,23.321250
4,2079.500000,37.138333,33.608333,33.623333,57.598333,27.671667,76.395000,32.320000,12.063333,17.600000
5,2374.285714,37.778571,30.101429,29.865714,55.075714,27.307143,75.848571,29.250000,11.562857,15.755714
6,2666.950000,42.816667,40.978333,41.093333,51.395000,26.555000,59.420000,35.595000,7.413333,20.426667
7,2980.100000,38.670000,46.914444,41.360000,48.993333,29.808889,53.730000,36.144444,8.281111,19.520000
8,2612.350000,41.850000,31.095000,31.187500,71.485000,21.365000,78.340000,29.292500,19.775000,12.387500
9,2670.350000,36.001667,31.685000,31.641667,59.063333,24.866667,69.350000,29.646667,11.155000,2.755000
